In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from datetime import date
import re
import pandas as pd

C:\Users\janva\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [7]:
url = f"https://www.gall.nl/bier/?start=0"
r = requests.get(url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
soup = BeautifulSoup(r.text, "html.parser")
soup


<!DOCTYPE html>

<html class="no-js is--v2" data-edit-mode="false" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<script src="https://cdn.optimizely.com/js/19212021254.js"></script>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="/on/demandware.static/Sites-gall-nl-Site/-/nl_NL/v1679324158207/js-external/cookie-consent.min.js" name="cookie-script"/>
<meta content="" name="client-log-level"/>
<script data-mopinion-deployment-id="2o8m5cnx2oxb82ji8ign87czqii5jwgsrwv" defer="" id="mopinion-script" src="//deploy.mopinion.com/js/pastease.js"></script>
<meta content="Bier bestellen? Bekijk onze bier aanbiedingen! | Gall &amp; Gall" property="og:title"/>
<meta content="website" property="og:type"/>
<meta content="https://www.gall.nl/bier/?start=0" property="og:url"/>
<meta content="Trek in een ijskoude bier? Bekijk onze aanbiedingen en koop jouw favoriete bier met een druk op de knop!" property="

In [7]:
"https://www.gall.nl" + soup.find_all("button", class_="pdp-details__item")[-1]['data-url']

'https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=141003'

In [8]:
products = soup.find_all('div', class_ ="product-grid__product")

In [24]:
float(products[6].find(class_ ="price-v2 ptile-v2_price")['aria-label'].replace("€ ", "").replace(" Per fles", ""))

ValueError: could not convert string to float: 'Van: 2.39 voor: 1.89'

In [15]:
products[2]

<div class="product-grid__product" itemprop="itemListElement" itemscope="" itemtype="https://schema.org/ListItem">
<meta content="2" itemprop="position">
<script type="text/javascript">//<!--
/* <![CDATA[ */
(function(){
try {
    if(window.CQuotient) {
	var cq_params = {};
	
	cq_params.cookieId = window.CQuotient.getCQCookieId();
	cq_params.userId = window.CQuotient.getCQUserId();
	cq_params.emailId = CQuotient.getCQHashedEmail();
	cq_params.loginId = CQuotient.getCQHashedLogin();
	cq_params.accumulate = true;
	cq_params.products = [{
	    id: '992739',
	    sku: ''
	}];
	cq_params.categoryId = '217';
	cq_params.refinements = '[{\"name\":\"Category\",\"value\":\"217\"}]';
	cq_params.personalized = 'false';
	cq_params.sortingRule = 'sort.dyna.hardlopers.orders';
	cq_params.imageUUID = '__UNDEFINED__';
	cq_params.realm = "BCLJ";
	cq_params.siteId = "gall-nl";
	cq_params.instanceType = "prd";
	cq_params.queryLocale = "nl_NL";
	cq_params.locale = window.CQuotient.locale;
	
	if(window.CQuo

In [20]:
products[3].find('p', class_ ="ptile-v2_desc").find_all('span')[1].text

'2022'

In [3]:
data = []
def retrieve_products(page_url, products, data):    
    for product in products:
        try:
            info = {
                'product' : len(data) + 1,
                'page_url' : page_url,
                'name' : product.find('strong', itemprop="name").text}

            try:
                info['product_link'] = "https://www.gall.nl" + product.find('a', class_="ptile-v2_link")['href']
            except:
                info['product_link'] = None
            try:
                info['type'] = product.find(class_ ="ptile-v2_profile").text.strip()
            except:
                info['type'] = None
            try:
                info['country'] = product.find('span', class_ ="ptile-v2_country").text
            except:
                info['country'] = None
            try:
                info['badge'] = product.find(class_ = "ptile-v2_badges").text.strip()
            except:
                info['badge'] = None
            try:
                info['price'] = float(product.find(class_ ="price-v2 ptile-v2_price")['aria-label'].replace("€ ", ""))
            except:
                info['price'] = None
            try:
                info['size'] = product.find('p', class_ ="ptile-v2_desc").find('i').text
            except:
                info['size'] = None
            try:
                info['rating'] = float(product.find('span' , class_ ="rating_label" ).text)
            except:
                info['rating'] = None
            try:
                info['count'] = int(product.find('span' , class_ ="rating_label")['data-count'])
            except:
                info['count'] = None
            try:
                info['year'] = product.find('p', class_ ="ptile-v2_desc").find_all('span')[1].text
            except:
                info['year'] = None
            print(info)
            data.append(info)
        except:
            pass

In [4]:
data = []
for i in range(0, 216, 12):
    page_url = "https://www.gall.nl/bier/?start=" + str(i)
    print(f"looking at {page_url}")
    r = requests.get(page_url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.text, "html.parser")
    products = soup.find_all('div', class_ ="product-grid__product")
    retrieve_products(page_url = page_url, products = products, data = data)
    print(len(data))
    time.sleep(10)

looking at https://www.gall.nl/bier/?start=0
{'product': 1, 'page_url': 'https://www.gall.nl/bier/?start=0', 'name': "Brouwerij 't IJ Cadeauverpakking", 'product_link': 'https://www.gall.nl/brouwerij-t-ij-cadeauverpakking-198cl-907693.html', 'type': 'Regionaal', 'country': None, 'badge': None, 'price': 17.99, 'size': '198CL', 'rating': 4.5, 'count': 11, 'year': None}
{'product': 2, 'page_url': 'https://www.gall.nl/bier/?start=0', 'name': 'Grolsch Lentebok', 'product_link': 'https://www.gall.nl/grolsch-lentebok-30cl-992739.html', 'type': 'Seizoen', 'country': None, 'badge': None, 'price': None, 'size': '30CL', 'rating': 4.3, 'count': 4, 'year': None}
{'product': 3, 'page_url': 'https://www.gall.nl/bier/?start=0', 'name': 'Bier&Box België', 'product_link': 'https://www.gall.nl/bier-box-belgie-133cl-908045.html', 'type': 'Geschenkverpakking', 'country': None, 'badge': None, 'price': 17.99, 'size': '133CL', 'rating': 4.8, 'count': 14, 'year': None}
{'product': 4, 'page_url': 'https://www.g

looking at https://www.gall.nl/bier/?start=36
{'product': 35, 'page_url': 'https://www.gall.nl/bier/?start=36', 'name': 'La Trappe Tripel', 'product_link': 'https://www.gall.nl/la-trappe-tripel-30cl-991635.html', 'type': 'Tripel', 'country': None, 'badge': None, 'price': None, 'size': '30CL', 'rating': 4.6, 'count': 8, 'year': None}
{'product': 36, 'page_url': 'https://www.gall.nl/bier/?start=36', 'name': 'La Chouffe', 'product_link': 'https://www.gall.nl/la-chouffe-75cl-990949.html', 'type': 'Blond', 'country': None, 'badge': None, 'price': 4.99, 'size': '75CL', 'rating': 4.9, 'count': 8, 'year': None}
{'product': 37, 'page_url': 'https://www.gall.nl/bier/?start=36', 'name': 'Heineken Mono', 'product_link': 'https://www.gall.nl/heineken-mono-12x25cl-996653.html', 'type': 'Pils', 'country': None, 'badge': None, 'price': 11.99, 'size': '12X25CL', 'rating': 3.0, 'count': 1, 'year': None}
{'product': 38, 'page_url': 'https://www.gall.nl/bier/?start=36', 'name': 'Duvel', 'product_link': 'h

looking at https://www.gall.nl/bier/?start=72
{'product': 71, 'page_url': 'https://www.gall.nl/bier/?start=72', 'name': 'Westmalle Tripel', 'product_link': 'https://www.gall.nl/westmalle-tripel-75cl-989088.html', 'type': 'Tripel', 'country': None, 'badge': None, 'price': 7.29, 'size': '75CL', 'rating': 4.7, 'count': 3, 'year': None}
{'product': 72, 'page_url': 'https://www.gall.nl/bier/?start=72', 'name': 'Hertog Jan Tripel', 'product_link': 'https://www.gall.nl/hertog-jan-tripel-30cl-910244.html', 'type': 'Tripel', 'country': None, 'badge': None, 'price': None, 'size': '30CL', 'rating': 3.8, 'count': 4, 'year': None}
{'product': 73, 'page_url': 'https://www.gall.nl/bier/?start=72', 'name': 'VandeStreek Playground - Non Alcoholic IPA', 'product_link': 'https://www.gall.nl/vandestreek-playground---non-alcoholic-ipa-33cl-147664.html', 'type': 'Alcoholvrij - Bier', 'country': None, 'badge': None, 'price': None, 'size': '33CL', 'rating': 5.0, 'count': 1, 'year': None}
{'product': 74, 'page

looking at https://www.gall.nl/bier/?start=108
{'product': 107, 'page_url': 'https://www.gall.nl/bier/?start=108', 'name': 'Bierbox 4vaks Alcoholvrij', 'product_link': 'https://www.gall.nl/bierbox-4vaks-alcoholvrij-170216.html', 'type': 'Geschenkverpakking', 'country': None, 'badge': None, 'price': 8.77, 'size': None, 'rating': None, 'count': None, 'year': None}
{'product': 108, 'page_url': 'https://www.gall.nl/bier/?start=108', 'name': 'Sport Zot', 'product_link': 'https://www.gall.nl/sport-zot-33cl-128120.html', 'type': 'Alcoholvrij / Alcoholarm', 'country': None, 'badge': None, 'price': None, 'size': '33CL', 'rating': 4.0, 'count': 1, 'year': None}
{'product': 109, 'page_url': 'https://www.gall.nl/bier/?start=108', 'name': 'Apple Bandit Cider Juicy Apple', 'product_link': 'https://www.gall.nl/apple-bandit-cider-juicy-apple-30cl-912093.html', 'type': 'Cider', 'country': None, 'badge': None, 'price': None, 'size': '30CL', 'rating': 4.5, 'count': 4, 'year': None}
{'product': 110, 'page

looking at https://www.gall.nl/bier/?start=144
{'product': 143, 'page_url': 'https://www.gall.nl/bier/?start=144', 'name': 'St.Christoffel Tripel', 'product_link': 'https://www.gall.nl/st.christoffel-tripel-33cl-127485.html', 'type': 'Tripel', 'country': None, 'badge': None, 'price': None, 'size': '33CL', 'rating': None, 'count': None, 'year': None}
{'product': 144, 'page_url': 'https://www.gall.nl/bier/?start=144', 'name': "O'Hara's Irish Stout", 'product_link': 'https://www.gall.nl/oharas-irish-stout-33cl-155489.html', 'type': 'Stout / Porter', 'country': None, 'badge': None, 'price': None, 'size': '33CL', 'rating': None, 'count': None, 'year': None}
{'product': 145, 'page_url': 'https://www.gall.nl/bier/?start=144', 'name': 'Erdinger Weisbrau Hefe', 'product_link': 'https://www.gall.nl/erdinger-weisbrau-hefe-50cl-904945.html', 'type': 'Weizen', 'country': None, 'badge': None, 'price': None, 'size': '50CL', 'rating': None, 'count': None, 'year': None}
{'product': 146, 'page_url': 'ht

looking at https://www.gall.nl/bier/?start=180
{'product': 179, 'page_url': 'https://www.gall.nl/bier/?start=180', 'name': 'Oedipus Thai Thai', 'product_link': 'https://www.gall.nl/oedipus-thai-thai-33cl-911240.html', 'type': 'Tripel', 'country': None, 'badge': None, 'price': None, 'size': '33CL', 'rating': 4.5, 'count': 2, 'year': None}
{'product': 180, 'page_url': 'https://www.gall.nl/bier/?start=180', 'name': 'Anchor Porter', 'product_link': 'https://www.gall.nl/anchor-porter-35.5cl-904503.html', 'type': 'Stout / Porter', 'country': None, 'badge': None, 'price': None, 'size': '35.5CL', 'rating': 4.5, 'count': 2, 'year': None}
{'product': 181, 'page_url': 'https://www.gall.nl/bier/?start=180', 'name': 'Wittekerke Rose', 'product_link': 'https://www.gall.nl/wittekerke-rose-8x25cl-973823.html', 'type': 'Fruit/Sours', 'country': None, 'badge': None, 'price': 10.32, 'size': '8X25CL', 'rating': None, 'count': None, 'year': None}
{'product': 182, 'page_url': 'https://www.gall.nl/bier/?star

In [5]:
pd.DataFrame(data).to_csv("gall_beer_links.csv", sep = ";")

In [131]:
[e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]

['Beschrijving', 'Druivensoort', 'Etiketinformatie', 'Prijswinnend']

In [134]:
soup.find_all('span', class_ ="pdp-details__item-label-secondary")[2].text.replace('\n', '')

'1 keer bekroond'

# Get product information

In [130]:
pos = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")].index("Prijswinnend")
soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')

IndexError: list index out of range

In [124]:
product_infos = []
def get_product_info(product_url, product_infos):
    r = requests.get(product_url, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(r.text, "html.parser")
    time.sleep(2)
    print(f"Currently looking at {product_url}")
    
    info = {'link' : product_url}
    try:
        info['type'] = soup.find('span', class_="pdp-info_profile").text.strip()
    except:
        info['type'] = None
    try:
        info['rating'] = float(soup.find('span', class_ ="rating_label").text)
    except:
        info['rating'] = None
    try:
        info['count'] = int(soup.find('span', class_ ="rating_label")['data-count'])
    except:
        info['count'] = None
    try:
        info['title'] = soup.find('h1', class_ ="pdp-info_name").text
    except:
        info['title'] = None
    try:
        info['country'] = soup.find('span', class_ = re.compile("flag-round u-flag-large u-hidden"))['title']
    except:
        info['country'] = None
    try:
        info['size'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[0].text
    except:
        info['size'] = None
    try:
        info['percentage'] = soup.find('p', class_ ="pdp-info_desc").find_all('i')[1].text
    except:
        info['percentage'] = None
    try:
        info['year'] = soup.find('p', class_ ="pdp-info_desc").find_all('span')[2].text
    except:
        info['year'] = None
    try:
        info['badge'] = soup.find('p', class_ ="pdp-info_badges").text.strip()
    except:
        info['badge'] = None
    try:
        info['price'] = float(soup.find('strong', class_ = "price-v2 pdp-info_price")['aria-label'].replace('€ ', ''))
    except:
        info['price'] = None
    try:
        info['main_description'] = soup.find('div', class_ ="pdp-info_short s-rich-text").text.strip()
    except:
        info['main_description'] = None
    try:
        usps = []
        for item in soup.find('ul', class_ = "m-usp-bar").find_all('div', class_="usp__text"):
            usps.append(item.text.strip())
        info['usps'] = usps
    except:
        info['usps'] = None
    
    try:
        info['secondary_description'] = soup.find('div', class_ = "pdp-details-description s-rich-text").text.strip()
    except:
        info['secondary_description'] = None
    details = [e.text for e in soup.find_all('span', class_ ="pdp-details__item-label")]
    try:
        pos = details.index("Smaak")
        info['taste'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['taste'] = None
    try:
        pos = details.index("Lekker bij")
        info['complementaries'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['complementaries'] = None
    try:
        pos = details.index("Druivensoort")
        info['grape'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['grape'] = None
    try:
        pos = details.index("Etiketinformatie")
        info['label'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['label'] = None
    try:
        pos = details.index("Prijswinnend")
        info['certificate'] = soup.find_all('span', class_ ="pdp-details__item-label-secondary")[pos].text.replace('\n', '')
    except:
        info['certificate'] = None
        
    try:
        info['producer_description'] = soup.find('div', class_="background-story__description s-rich-text read-more__text").text.strip()
    except:
        info['producer_description'] = None
    try:
        info['producer'] = soup.find('span', class_="background-story__name").text
    except:
        info['producer'] = None
    missing_values = [key for key,value in info.items() if value == None]
    if len(missing_values) > 0:
        print(f"the following items were missing: {missing_values}")
    product_infos.append(info)

In [122]:
get_product_info("https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html", product_infos)
product_infos

Currently looking at https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html


[{'link': 'https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html',
  'type': 'Fruit Lover',
  'rating': 4.4,
  'count': 199,
  'title': 'Mucho Más Blanco',
  'country': 'Spanje',
  'size': '75CL',
  'percentage': '12,5%',
  'year': None,
  'badge': '2e halve prijs',
  'price': 6.99,
  'main_description': 'Het Spaanse Castilla-La Mancha is het grootste wijngebied ter wereld. Van verdejo, sauvignon blanc en chardonnay maakt men ter plaatse deze veelzijdige witte wijn. Ideaal voor bij tapas.\n*Maximaal 6 flessen per klant.',
  'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
   'Gratis bezorging vanaf € 50,-',
   'Veilig betalen'],
  'secondary_description': 'Het doel van deze wijn was om een onderscheidende stijl te creëren door de kwaliteiten van drie verschillende druivensoorten (verdejo, sauvignon blanc en chardonnay) te benutten. De druiven fermenteren in roestvrijstalen tanks, de chardonnay rijpt daarna nog op een Frans eikenhouten vat. Dit alles zorgt voor een frisse wijn 

In [106]:
product_pages = pd.read_csv("gall_product_links.csv", sep = ";", index_col = 0)
product_pages

,product,page_url,name,product_link,type,country,badge,price,size,rating,count,year
0,1,https://www.gall.nl/wijn/?start=0,Mucho Más Tinto,https://www.gall.nl/mucho-mas-tinto-rood-75cl-...,Easy Rider,Spanje,2e halve prijs,6.99,75CL,4.6,200.0,NaN
1,2,https://www.gall.nl/wijn/?start=0,Mucho Más Blanco,https://www.gall.nl/mucho-mas-blanco-wit-75cl-...,Fruit Lover,Spanje,2e halve prijs,6.99,75CL,4.4,199.0,NaN
2,3,https://www.gall.nl/wijn/?start=0,La Palma Chardonnay,https://www.gall.nl/la-palma-chardonnay-wit-75...,Big White,Chili,2e halve prijs,5.99,75CL,4.4,200.0,2022.0
3,4,https://www.gall.nl/wijn/?start=0,Ogio Pinot Grigio,https://www.gall.nl/ogio-pinot-grigio-wit-75cl...,Fruit Lover,Italië,2e halve prijs,6.59,75CL,4.1,188.0,2022.0
4,5,https://www.gall.nl/wijn/?start=0,Valdivieso Merlot,https://www.gall.nl/valdivieso-merlot-rood-75c...,Easy Rider,Chili,2e halve prijs,6.59,75CL,4.3,199.0,2021.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1794,1795,https://www.gall.nl/wijn/?start=1788,Connétable de Talbot,https://www.gall.nl/connetable-de-talbot-rood-...,Royal Red,Frankrijk,2e halve prijs,32.99,75CL,NaN,NaN,2018.0
1795,1796,https://www.gall.nl/wijn/?start=1788,Umani Ronchi Cúmaro Conero Riserva,https://www.gall.nl/umani-ronchi-c%C3%BAmaro-c...,Classic Red,Italië,2e halve prijs,30.59,75CL,NaN,NaN,2016.0
1796,1797,https://www.gall.nl/wijn/?start=1800,Penfolds RWT Barossa Valley Shiraz,https://www.gall.nl/penfolds-rwt-barossa-valle...,Big Red,Australië,NaN,163.19,75CL,NaN,NaN,2016.0
1797,1798,https://www.gall.nl/wijn/?start=1800,Château Barde-Haut,https://www.gall.nl/ch%C3%A2teau-barde-haut-ro...,Royal Red,Frankrijk,2e halve prijs,54.49,75CL,NaN,NaN,2016.0


In [125]:
product_infos = []
for product_url in product_pages['product_link']:
    get_product_info(product_url, product_infos)
    print(len(product_infos))

Currently looking at https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html
the following items were missing: ['year']
1
Currently looking at https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html
the following items were missing: ['year', 'grape']
2
Currently looking at https://www.gall.nl/la-palma-chardonnay-wit-75cl-388017.html
3
Currently looking at https://www.gall.nl/ogio-pinot-grigio-wit-75cl-259381.html
4
Currently looking at https://www.gall.nl/valdivieso-merlot-rood-75cl-387622.html
5
Currently looking at https://www.gall.nl/alamos-malbec-rood-75cl-387312.html
6
Currently looking at https://www.gall.nl/bruce-jack-sauvignon-blanc-wit-75cl-145122.html
7
Currently looking at https://www.gall.nl/cap-royal-cuvee-tradition-rood-75cl-415138.html
8
Currently looking at https://www.gall.nl/la-palma-merlot-rood-75cl-388025.html
9
Currently looking at https://www.gall.nl/la-palma-reserva-merlot-rood-75cl-387835.html
10
Currently looking at https://www.gall.nl/valdivieso-sauvignon-

Currently looking at https://www.gall.nl/marques-de-gri%C3%B1on-rioja-reserva-rood-75cl-408867.html
79
Currently looking at https://www.gall.nl/faber-cabernet-sauvignon-rood-75cl-802085.html
the following items were missing: ['year', 'badge', 'price', 'certificate']
80
Currently looking at https://www.gall.nl/mucho-mas-tinto-bag-in-box-rood-300cl-139289.html
the following items were missing: ['year']
81
Currently looking at https://www.gall.nl/dom%C3%A4ne-wachau-chardonnay-federspiel-terrassen-wit-75cl-107611.html
the following items were missing: ['rating', 'count']
82
Currently looking at https://www.gall.nl/piccini-buon-governo-rosso-toscana-rood-75cl-101923.html
83
Currently looking at https://www.gall.nl/bodega-garzon-tannat-de-corte-rood-75cl-106763.html
84
Currently looking at https://www.gall.nl/natura-sauvignon-blanc-organic-white-wit-75cl-381926.html
85
Currently looking at https://www.gall.nl/vi%C3%B1a-mayor-rueda-wit-75cl-308811.html
86
Currently looking at https://www.gall

Currently looking at https://www.gall.nl/beyerskloof-synergy-cape-blend-rood-75cl-384593.html
153
Currently looking at https://www.gall.nl/la-capitana-single-vineyard-cabernet-franc-rood-75cl-402257.html
the following items were missing: ['badge']
154
Currently looking at https://www.gall.nl/el-parador-crianza-navarra-rood-75cl-309168.html
155
Currently looking at https://www.gall.nl/faber-rose-rose-75cl-802050.html
the following items were missing: ['year', 'badge', 'price', 'certificate']
156
Currently looking at https://www.gall.nl/bodega-garzon-estate-sauvignon-blanc-wit-75cl-106666.html
157
Currently looking at https://www.gall.nl/dows-fine-tawny-rood-75cl-460451.html
the following items were missing: ['year', 'grape']
158
Currently looking at https://www.gall.nl/echo-sauvignon-blanc-wit-75cl-163821.html
159
Currently looking at https://www.gall.nl/riporta-pecorino-wit-75cl-154709.html
the following items were missing: ['grape']
160
Currently looking at https://www.gall.nl/eisberg

Currently looking at https://www.gall.nl/errazuriz-aconcague-costa-sauvignon-blanc-wit-75cl-414336.html
224
Currently looking at https://www.gall.nl/trapiche-reserve-chardonnay-wit-75cl-382469.html
225
Currently looking at https://www.gall.nl/vendanges-mademoiselles-blanc-bio-wit-75cl-139351.html
the following items were missing: ['year', 'badge', 'price', 'grape', 'certificate']
226
Currently looking at https://www.gall.nl/ch%C3%A2teau-maillard-graves-rood-75cl-134384.html
227
Currently looking at https://www.gall.nl/lourensford-limited-release-merlot-rood-75cl-148466.html
the following items were missing: ['rating', 'count', 'taste', 'complementaries']
228
Currently looking at https://www.gall.nl/cune-rioja-gran-reserva-rood-75cl-308196.html
229
Currently looking at https://www.gall.nl/mucho-mas-blanco-bib-wit-300cl-408794.html
the following items were missing: ['year', 'secondary_description', 'certificate']
230
Currently looking at https://www.gall.nl/baumgartner-blauer-zweigelt-ro

Currently looking at https://www.gall.nl/penfolds-koonunga-hill-shiraz-rood-75cl-126691.html
290
Currently looking at https://www.gall.nl/cabolani-chardonnay-friuli-aquileia-wit-75cl-122742.html
the following items were missing: ['rating', 'count', 'taste', 'complementaries']
291
Currently looking at https://www.gall.nl/san-silvestro-barbera-dasti-superiore-rood-75cl-161616.html
292
Currently looking at https://www.gall.nl/tokara-reserve-collection-chardonnay-wit-75cl-380040.html
the following items were missing: ['badge', 'certificate']
293
Currently looking at https://www.gall.nl/faber-sauvignon-blanc-wit-75cl-400114.html
the following items were missing: ['year', 'badge', 'price', 'certificate']
294
Currently looking at https://www.gall.nl/ch%C3%A2teau-hautes-terres-bordeaux-blanc-wit-75cl-145084.html
295
Currently looking at https://www.gall.nl/torres-natureo-white-wit-75cl-802816.html
the following items were missing: ['badge', 'price', 'grape']
296
Currently looking at https://ww

...

Currently looking at https://www.gall.nl/falesco-syrah-tellus-rood-75cl-257087.html
1572
Currently looking at https://www.gall.nl/terre-di-verona-pinot-grigio-wit-75cl-162264.html
1573
Currently looking at https://www.gall.nl/tenuta-sette-ponti-oreno-rood-75cl-409642.html
the following items were missing: ['rating', 'count']
1574
Currently looking at https://www.gall.nl/joseph-drouhin-chablis-mont-de-milieu-wit-75cl-122661.html
the following items were missing: ['badge', 'taste']
1575
Currently looking at https://www.gall.nl/glorioso-rioja-crianza-rood-75cl-308757.html
the following items were missing: ['rating', 'count']
1576
Currently looking at https://www.gall.nl/lanzerac-merlot-rood-75cl-383600.html
the following items were missing: ['certificate']
1577
Currently looking at https://www.gall.nl/mommessin-fleurie-grandes-mises-rood-75cl-122955.html
the following items were missing: ['taste', 'complementaries']
1578
Currently looking at https://www.gall.nl/la-chablisienne-chablis-bla

Currently looking at https://www.gall.nl/la-chapelle-de-haut-bages-liberal-rood-75cl-155764.html
1625
Currently looking at https://www.gall.nl/bodegas-roda-1-reserva-magnum-rood-150cl-408085.html
the following items were missing: ['rating', 'count']
1626
Currently looking at https://www.gall.nl/la-chablisienne-chablis-premier-cru-les-lys-wit-75cl-405051.html
the following items were missing: ['rating', 'count', 'certificate']
1627
Currently looking at https://www.gall.nl/ch%C3%A2teau-troplong-mondot-rood-75cl-132691.html
the following items were missing: ['rating', 'count', 'certificate']
1628
Currently looking at https://www.gall.nl/ch%C3%A2teau-boutillot-rood-75cl-141836.html
the following items were missing: ['taste', 'complementaries']
1629
Currently looking at https://www.gall.nl/catena-adrianna-malbec-fortuna-terrae-rood-75cl-100374.html
1630
Currently looking at https://www.gall.nl/dannona-barbera-dasti-superiore-magnum-rood-150cl-122866.html
the following items were missing: ['

Currently looking at https://www.gall.nl/ch%C3%A2teau-conseillante-rood-75cl-132187.html
the following items were missing: ['rating', 'count', 'certificate']
1680
Currently looking at https://www.gall.nl/alain-brumont-ch%C3%A2teau-montus-xl-rood-75cl-140058.html
the following items were missing: ['taste', 'complementaries']
1681
Currently looking at https://www.gall.nl/winzer-krems-sommerrain-gr%C3%BCner-veltliner-sp%C3%A4tlese-wit-75cl-219320.html
1682
Currently looking at https://www.gall.nl/sansilvestro-barolo-del-comune-di-barolo-rood-75cl-157635.html
1683
Currently looking at https://www.gall.nl/ch%C3%A2teau-doisy-daene-wit-75cl-139297.html
the following items were missing: ['rating', 'count']
1684
Currently looking at https://www.gall.nl/joseph-drouhin-chassagne-montrachet-wit-75cl-169439.html
the following items were missing: ['badge']
1685
Currently looking at https://www.gall.nl/bodegas-la-horra-corimbo-1-rood-75cl-408069.html
the following items were missing: ['rating', 'coun

Currently looking at https://www.gall.nl/antinori-tignanello-rood-75cl-256633.html
the following items were missing: ['badge', 'certificate']
1733
Currently looking at https://www.gall.nl/ch%C3%A2teau-la-fleur-de-gay-rood-75cl-132284.html
the following items were missing: ['rating', 'count']
1734
Currently looking at https://www.gall.nl/chablisienne-chablis-lhomme-mort-wit-75cl-405086.html
the following items were missing: ['certificate']
1735
Currently looking at https://www.gall.nl/ch%C3%A2teau-leglise-clinet-rood-75cl-132780.html
the following items were missing: ['rating', 'count', 'badge']
1736
Currently looking at https://www.gall.nl/ch%C3%A2teau-canon-rood-75cl-132357.html
the following items were missing: ['rating', 'count', 'badge']
1737
Currently looking at https://www.gall.nl/alain-brumont-ch%C3%A2teau-montus-la-tyre-rood-75cl-140236.html
the following items were missing: ['rating', 'count', 'badge', 'taste', 'complementaries']
1738
Currently looking at https://www.gall.nl/t

Currently looking at https://www.gall.nl/alambrado-cabernet-franc-rood-75cl-413941.html
the following items were missing: ['rating', 'count']
1784
Currently looking at https://www.gall.nl/canallas-rose-rose-75cl-414972.html
the following items were missing: ['rating', 'count', 'producer_description', 'producer']
1785
Currently looking at https://www.gall.nl/zuccardi-aluvional-gualtallary--rood-75cl-414107.html
the following items were missing: ['rating', 'count']
1786
Currently looking at https://www.gall.nl/zuccardi-los-oscuros-red-blend---rood-75cl-414158.html
the following items were missing: ['rating', 'count', 'certificate']
1787
Currently looking at https://www.gall.nl/zuccardi-aluvional-altamira--rood-75cl-414077.html
the following items were missing: ['rating', 'count']
1788
Currently looking at https://www.gall.nl/zuccardi-aluvional-los-chacayes--rood-75cl-414069.html
the following items were missing: ['rating', 'count']
1789
Currently looking at https://www.gall.nl/zuccardi-c

In [126]:
pd.DataFrame(product_infos).to_csv("gall_product_data.csv", sep = ";")

In [137]:
pd.DataFrame(product_infos).to_csv("gall_product_data_utf16.csv", sep = ";", encoding = 'utf-8-sig')

In [135]:
product_infos[32]

{'link': 'https://www.gall.nl/baumgartner-gr%C3%BCner-veltliner-wit-75cl-219819.html',
 'type': 'Fruit Lover',
 'rating': 4.2,
 'count': 75,
 'title': 'Baumgartner Grüner Veltliner',
 'country': 'Oostenrijk',
 'size': '75CL',
 'percentage': '12,5%',
 'year': '2021',
 'badge': '2e halve prijs',
 'price': 7.29,
 'main_description': 'Grüner veltliner is de nationale druif van Oostenrijk. Meer dan 30% van de wijngaarden in Oostenrijk staan aangeplant met de druif die in Engeland ook wel ‘groovy’ wordt genoemd.',
 'usps': ['Bestel voor 22:00 vandaag, morgen in huis',
  'Gratis bezorging vanaf € 50,-',
  'Veilig betalen'],
 'secondary_description': "Grüner veltliner is de nationale druif van Oostenrijk. Meer dan 30% van het wijngaardareaal in dit Alpenland staat aangeplant met de druif die in Engeland ook wel 'groovy' wordt genoemd. Deze wijn van Baumgartner komt uit de regio Niederösterreich. Hij is fris, kruidig en aromatisch met gele appel, peer, witte peper en limoen. Grüner veltliner is

# Awards

In [ ]:
"https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id="

In [90]:
product_pages = pd.read_csv("gall_product_data.csv", sep = ";", index_col = 0)

In [18]:
for link in product_pages['product_link'][0:10]:
    identity = link.split('-')[-1].replace('.html', '')
    print("https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=" +identity)

https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=117196
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=139939
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=388017
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=259381
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=387622
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=387312
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=145122
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=415138
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=388025
https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=387835


In [104]:
for index, row in product_pages.iterrows():
    print(f"{counter}/{len(product_pages)} : {row['link']}")
    counter += 1
    if ('bekroond' in str(row['label'])) or ('bekroond' in str(row['certificate'])):
        pass

171/1799 : https://www.gall.nl/mucho-mas-tinto-rood-75cl-117196.html
172/1799 : https://www.gall.nl/mucho-mas-blanco-wit-75cl-139939.html
173/1799 : https://www.gall.nl/la-palma-chardonnay-wit-75cl-388017.html
174/1799 : https://www.gall.nl/ogio-pinot-grigio-wit-75cl-259381.html
175/1799 : https://www.gall.nl/valdivieso-merlot-rood-75cl-387622.html
176/1799 : https://www.gall.nl/alamos-malbec-rood-75cl-387312.html
177/1799 : https://www.gall.nl/bruce-jack-sauvignon-blanc-wit-75cl-145122.html
178/1799 : https://www.gall.nl/cap-royal-cuvee-tradition-rood-75cl-415138.html
179/1799 : https://www.gall.nl/la-palma-merlot-rood-75cl-388025.html
180/1799 : https://www.gall.nl/la-palma-reserva-merlot-rood-75cl-387835.html
181/1799 : https://www.gall.nl/valdivieso-sauvignon-blanc-wit-75cl-387630.html
182/1799 : https://www.gall.nl/piccini-memoro-rosso-rood-75cl-100803.html
183/1799 : https://www.gall.nl/valdivieso-chardonnay-wit-75cl-387614.html
184/1799 : https://www.gall.nl/la-palma-reserva-car

637/1799 : https://www.gall.nl/errazuriz-aconcagua-costa-pinot-noir-rood-75cl-409596.html
638/1799 : https://www.gall.nl/san-silvestro-barbera-dalba-superiore-rood-75cl-161586.html
639/1799 : https://www.gall.nl/cabolani-pinot-grigio-friuli-aquileia-wit-75cl-122750.html
640/1799 : https://www.gall.nl/cune-rioja-reserva-rood-37.5cl-139580.html
641/1799 : https://www.gall.nl/artis-rose-rose-75cl-404128.html
642/1799 : https://www.gall.nl/joseph-drouhin-pouilly-vinzelles-wit-75cl-169609.html
643/1799 : https://www.gall.nl/alamos-malbec-magnum-rood-150cl-125075.html
644/1799 : https://www.gall.nl/winzer-krems-sandgrube-13-kremstal-blauer-zweigelt-rose-rose-75cl-219916.html
645/1799 : https://www.gall.nl/rocca-sveva-nottefonda-rood-75cl-257338.html
646/1799 : https://www.gall.nl/zorzal-gran-terroir-cabernet-sauvignon-rood-75cl-163058.html
647/1799 : https://www.gall.nl/ascheri-gavi-comune-di-gavi-cristina-wit-75cl-123056.html
648/1799 : https://www.gall.nl/san-silvestro-piemonte-barbera-bio

1180/1799 : https://www.gall.nl/renato-ratti-barolo-marcenasco-rood-75cl-123919.html
1181/1799 : https://www.gall.nl/falesco-poggio-dei-gelsi-bianco-lazio-wit-75cl-257095.html
1182/1799 : https://www.gall.nl/librandi-gravello-rosso-rood-75cl-110744.html
1183/1799 : https://www.gall.nl/la-sirene-de-giscours-rood-75cl-415529.html
1184/1799 : https://www.gall.nl/antinori-conte-della-vipera-sauvignon-wit-75cl-256811.html
1185/1799 : https://www.gall.nl/la-chablisienne-chablis-grenouilles-wit-75cl-122459.html
1186/1799 : https://www.gall.nl/m.-grinon-valdepusa-cabernet-sauvignon-rood-75cl-408891.html
1187/1799 : https://www.gall.nl/ch%C3%A2teau-cos-destournel-rood-75cl-416053.html
1188/1799 : https://www.gall.nl/maison-ventenac-les-dissidents-paul-rood-75cl-143006.html
1189/1799 : https://www.gall.nl/raymond-generations-cabernet-sauvignon-rood-75cl-115967.html
1190/1799 : https://www.gall.nl/ch%C3%A2teau-la-pointe-rood-75cl-133973.html
1191/1799 : https://www.gall.nl/domaine-drouhin-roseroc

In [108]:
data = []
#with open("gall_awards.csv", "w", encoding = "utf-8-sig", newline = '') as csv_file:
    #writer = csv.writer(csv_file, delimiter = ";")
    #writer.writerow(['product_link', 'award_link', 'vintage', 'grade', 'contributor', ])
counter = 0
for index, row in product_pages.iloc[63:,].iterrows():
    print(f"{counter}/{len(product_pages)} : {row['link']}")
    counter += 1
    if ('bekroond' in str(row['label'])) or ('bekroond' in str(row['certificate'])):
        product_link = row['link']
        identity = product_link.split('-')[-1].replace('.html', '')
        award_link = "https://www.gall.nl/on/demandware.store/Sites-gall-nl-Site/nl_NL/Product-Awards?&id=" +identity
        r = requests.get(award_link, headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"})
        soup = BeautifulSoup(r.text, "html.parser")
        time.sleep(2)
        retrieve_award_info(product_link = product_link, award_link = award_link, soup = soup, data = data)
    

0/1799 : https://www.gall.nl/domaine-andau-gr%C3%BCner-veltliner-wit-75cl-219851.html
1
2
3
4
5
6
7
8
1/1799 : https://www.gall.nl/lawsons-dry-hills-riesling-wit-75cl-121304.html
9
10
11
2/1799 : https://www.gall.nl/ironstone-chardonnay-wit-75cl-388734.html
12
13
14
3/1799 : https://www.gall.nl/torres-vi%C3%B1a-sol-garnacha-rood-75cl-118877.html
15
16
17
4/1799 : https://www.gall.nl/vi%C3%B1a-salceda-rueda-verdejo-wit-75cl-131814.html
18
19
20
5/1799 : https://www.gall.nl/errazuriz-estate-cabernet-sauvignon-reserva-rood-75cl-409448.html
21
22
6/1799 : https://www.gall.nl/mountain-vineyards-sauvignon-blanc-wit-75cl-381764.html
23
24
25
7/1799 : https://www.gall.nl/les-hautes-pentes-rose-rose-75cl-409499.html
26
27
28
8/1799 : https://www.gall.nl/domaine-andau-zweigelt-rood-75cl-219266.html
29
30
31
32
33
34
9/1799 : https://www.gall.nl/yealands-sauvignon-blanc-wit-75cl-383570.html
35
36
37
38
39
40
41
42
43
10/1799 : https://www.gall.nl/arrogant-frog-sauvignon-blanc-wit-75cl-129372.html

316
317
318
319
320
89/1799 : https://www.gall.nl/beyerskloof-synergy-cape-blend-rood-75cl-384593.html
321
322
323
324
325
326
327
328
329
90/1799 : https://www.gall.nl/la-capitana-single-vineyard-cabernet-franc-rood-75cl-402257.html
330
91/1799 : https://www.gall.nl/el-parador-crianza-navarra-rood-75cl-309168.html
331
332
333
92/1799 : https://www.gall.nl/faber-rose-rose-75cl-802050.html
93/1799 : https://www.gall.nl/bodega-garzon-estate-sauvignon-blanc-wit-75cl-106666.html
334
335
336
94/1799 : https://www.gall.nl/dows-fine-tawny-rood-75cl-460451.html
337
95/1799 : https://www.gall.nl/echo-sauvignon-blanc-wit-75cl-163821.html
338
339
96/1799 : https://www.gall.nl/riporta-pecorino-wit-75cl-154709.html
340
341
97/1799 : https://www.gall.nl/eisberg-sauvignon-blanc-wit-75cl-101745.html
98/1799 : https://www.gall.nl/inycon-growers-pinot-grigio-blush-rose-75cl-100676.html
342
343
344
99/1799 : https://www.gall.nl/yealands-sauvignon-blanc-blush-rose-rose-75cl-379867.html
345
346
100/1799 : 

617
618
619
620
176/1799 : https://www.gall.nl/duo-pakket-italie-van-tophuis-antinori-gemengd-2x75cl-413259.html
177/1799 : https://www.gall.nl/mulderbosch-steen-op-hout-wit-75cl-113530.html
621
622
623
178/1799 : https://www.gall.nl/van-loveren-sauvignon-blanc-wit-75cl-382671.html
624
625
626
627
628
179/1799 : https://www.gall.nl/yealands-estate-single-block-m2-organic-wit-75cl-381861.html
629
630
631
632
633
180/1799 : https://www.gall.nl/bodega-garzon-reserva-cabernet-franc-rood-75cl-106623.html
634
635
636
637
181/1799 : https://www.gall.nl/bodega-garzon-estate-pinot-grigio-wit-75cl-106534.html
638
639
640
182/1799 : https://www.gall.nl/errazuriz-aconcagua-costa-chardonnay-wit-75cl-414328.html
641
642
183/1799 : https://www.gall.nl/lawsons-dry-hills-pinot-noir-rood-75cl-121002.html
643
644
184/1799 : https://www.gall.nl/arrogant-frog-rose-bag-in-box-rose-300cl-138711.html
185/1799 : https://www.gall.nl/errazuriz-aconcagua-alto-carmenere-rood-75cl-409588.html
645
646
186/1799 : htt

884
262/1799 : https://www.gall.nl/villa-locatelli-cabernet-franc-rood-75cl-258903.html
263/1799 : https://www.gall.nl/dom%C3%A4ne-wachau-federspiel-liebenberg-wit-75cl-218995.html
885
886
887
264/1799 : https://www.gall.nl/catena-alta-cabernet-sauvignon-rood-75cl-382019.html
888
889
890
891
892
893
894
895
265/1799 : https://www.gall.nl/la-route-des-plages-rose-75cl-164488.html
896
897
266/1799 : https://www.gall.nl/duo-pakket-rioja-gran-reserva-rood-2x75cl-413291.html
267/1799 : https://www.gall.nl/trapiche-pinot-grigio-wit-75cl-382590.html
898
899
900
268/1799 : https://www.gall.nl/casal-mendes-vinho-verde-wit-75cl-353027.html
269/1799 : https://www.gall.nl/valdivieso-single-vineyard-syrah-rood-75cl-110191.html
901
902
270/1799 : https://www.gall.nl/karl-kaspar-auslese-wit-75cl-219991.html
271/1799 : https://www.gall.nl/maison-ventenac-reserve-de-jeanne-rood-75cl-143022.html
903
904
905
906
272/1799 : https://www.gall.nl/nice-rose-rose-75cl-409340.html
907
908
273/1799 : https://www

1115
1116
1117
1118
1119
347/1799 : https://www.gall.nl/torres-celeste-verdejo-wit-75cl-402362.html
1120
348/1799 : https://www.gall.nl/casal-mendes-rose-rose-75cl-353019.html
1121
349/1799 : https://www.gall.nl/lourensford-chrysalis-white-wit-75cl-148458.html
1122
1123
350/1799 : https://www.gall.nl/andre-stuber-riesling-wit-75cl-140074.html
1124
351/1799 : https://www.gall.nl/villa-locatelli-pinot-bianco-wit-75cl-258865.html
1125
1126
1127
352/1799 : https://www.gall.nl/eagles-nest-merlot-magnum-rood-150cl-122831.html
1128
1129
1130
353/1799 : https://www.gall.nl/les-courtelles-picpoul-de-pinet-wit-75cl-196770.html
1131
1132
1133
354/1799 : https://www.gall.nl/gruber-roschitz-riesling-roschitz-wit-75cl-110671.html
1134
1135
355/1799 : https://www.gall.nl/deloach-california-pinot-noir-rood-75cl-112194.html
1136
356/1799 : https://www.gall.nl/mirabeau-classic-rose-geschenkverpakking-rose-75cl-117781.html
1137
1138
1139
357/1799 : https://www.gall.nl/freetime-merlot-rood-75cl-117684.htm

...

Error at 0
1352/1799 : https://www.gall.nl/alcoholvrijpakket-try-januari--wit-400cl-161187.html
1353/1799 : https://www.gall.nl/torres-vi%C3%B1a-sol-wit-75cl-315052.html
3849
3850
1354/1799 : https://www.gall.nl/alamos-cabernet-sauvignon-rood-75cl-387789.html
3851
3852
3853
3854
3855
3856
1355/1799 : https://www.gall.nl/bruce-jack-reserve-pinotage-rood-75cl-157716.html
3857
3858
1356/1799 : https://www.gall.nl/oveja-negra-winemakers-selection-malbec-petit-verd-rood-75cl-382507.html
3859
3860
3861
3862
3863
1357/1799 : https://www.gall.nl/lawsons-dry-hills-pinot-noir-rose-rose-75cl-121185.html
3864
3865
3866
1358/1799 : https://www.gall.nl/tokara-shiraz-rood-75cl-380369.html
3867
3868
3869
3870
1359/1799 : https://www.gall.nl/valdivieso-winemakers-reserva-malbec-rood-75cl-127582.html
3871
3872
1360/1799 : https://www.gall.nl/alamos-viognier-wit-75cl-388661.html
3873
3874
3875
3876
3877
1361/1799 : https://www.gall.nl/inycon-estate-chardonnay-viognier-wit-75cl-259748.html
1362/1799 : htt

4036
4037
4038
1436/1799 : https://www.gall.nl/m%C3%A2con-villages-wit-75cl-404160.html
4039
1437/1799 : https://www.gall.nl/tokara-reserve-collection-syrah-rood-75cl-126500.html
4040
4041
4042
1438/1799 : https://www.gall.nl/castello-di-albola-chardonnay-toscana-wit-75cl-122270.html
4043
4044
1439/1799 : https://www.gall.nl/laurus-aop-chateauneuf-rood-75cl-140392.html
4045
4046
4047
4048
1440/1799 : https://www.gall.nl/natura-merlot-organic-red-rood-75cl-381918.html
4049
1441/1799 : https://www.gall.nl/brio-de-cantenac-brown-rood-75cl-126969.html
4050
4051
4052
1442/1799 : https://www.gall.nl/boschkloof-epilogue-rood-75cl-391131.html
4053
4054
4055
Error at 9
1443/1799 : https://www.gall.nl/trimbach-pinot-blanc-wit-75cl-104531.html
4056
4057
4058
4059
4060
1444/1799 : https://www.gall.nl/mommessin-c%C3%B4te-de-brouilly-grandes-mises-rood-75cl-179167.html
4061
4062
1445/1799 : https://www.gall.nl/j.-de-villebois-sancerre-blanc-wit-75cl-106844.html
4063
4064
4065
4066
4067
4068
4069
407

4246
1518/1799 : https://www.gall.nl/brocard-bourgogne-chardonnay-wit-75cl-150703.html
1519/1799 : https://www.gall.nl/penfolds-bin-138-shiraz-grenache-mataro-rood-75cl-407917.html
4247
4248
1520/1799 : https://www.gall.nl/j.-de-villebois-sancerre-blanc-magnum-wit-150cl-123250.html
4249
4250
4251
1521/1799 : https://www.gall.nl/pouilly-fume-wit-75cl-144584.html
1522/1799 : https://www.gall.nl/catena-malbec-magnum-rood-150cl-124834.html
4252
4253
4254
4255
4256
1523/1799 : https://www.gall.nl/mathilde-chapoutier-grand-ferrage-rose-rose-75cl-111686.html
4257
4258
4259
1524/1799 : https://www.gall.nl/brocard-chablis-valmur-wit-75cl-150711.html
4260
1525/1799 : https://www.gall.nl/fritz-wassmer-schlossberg-staufen-chardonnay-wit-75cl-404497.html
4261
4262
1526/1799 : https://www.gall.nl/trapiche-medalla-chardonnay-wit-75cl-404772.html
4263
4264
1527/1799 : https://www.gall.nl/lavis-d%C3%BCrer-weg-lagrein-rood-75cl-142387.html
1528/1799 : https://www.gall.nl/haut-medoc-de-giscours-rood-75cl

4450
4451
4452
1602/1799 : https://www.gall.nl/domaine-clos-maurice-saumur-rood-75cl-159166.html
4453
1603/1799 : https://www.gall.nl/ch%C3%A2teau-haut-bages-liberal-rood-75cl-132144.html
4454
4455
1604/1799 : https://www.gall.nl/j.c.-boisset-m%C3%A2con-villages-vieilles-vignes-wit-75cl-169501.html
4456
1605/1799 : https://www.gall.nl/mathilde-chapoutier-rueda-wit-75cl-111651.html
1606/1799 : https://www.gall.nl/le-cloitre-de-prieure-lichine-rood-75cl-114693.html
4457
1607/1799 : https://www.gall.nl/tenimenti-dalessandro-bosco-syrah-rood-75cl-100447.html
4458
4459
4460
1608/1799 : https://www.gall.nl/le-vigne-di-zam%C3%B2-refosco-rood-75cl-258431.html
4461
4462
4463
4464
Error at 12
1609/1799 : https://www.gall.nl/joseph-drouhin-chablis-premier-cru-secher-wit-75cl-169919.html
4465
4466
1610/1799 : https://www.gall.nl/ch%C3%A2teau-chasse-spleen-rood-75cl-134872.html
4467
4468
4469
4470
1611/1799 : https://www.gall.nl/jc-boisset-pommard-les-arvelets-rood-75cl-122823.html
4471
1612/1799 :

4657
1687/1799 : https://www.gall.nl/domaine-de-lamaurigue-rose-methusalem-rose-600cl-117617.html
4658
4659
1688/1799 : https://www.gall.nl/ch%C3%A2teau-valandraud-rood-75cl-160113.html
4660
4661
4662
1689/1799 : https://www.gall.nl/penfolds-grange-rood-75cl-382965.html
4663
4664
4665
4666
4667
4668
1690/1799 : https://www.gall.nl/vieux-ch%C3%A2teau-certan-rood-75cl-133795.html
4669
4670
4671
4672
4673
4674
1691/1799 : https://www.gall.nl/ch%C3%A2teau-calon-segur-rood-75cl-117080.html
4675
4676
4677
4678
1692/1799 : https://www.gall.nl/ch%C3%A2teau-la-mission-haut-brion-rood-75cl-124443.html
4679
1693/1799 : https://www.gall.nl/ch%C3%A2teau-mouton-rothschild-rood-75cl-156930.html
4680
4681
4682
1694/1799 : https://www.gall.nl/piccini-memoro-rosso-magnum-rood-150cl-154032.html
4683
4684
4685
1695/1799 : https://www.gall.nl/duo-pakket-super-tuscan-rood-2x75cl-413305.html
1696/1799 : https://www.gall.nl/e.guigal-c%C3%B4te-rotie-la-landonne-rood-75cl-119784.html
4686
4687
1697/1799 : https

In [65]:
text = soup.find_all(text = True)
text

['\n',
 'Prijswinnend',
 '\n',
 '2 keer bekroond',
 '\n',
 '\n',
 'Oogstjaar 0000',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n90 punten\n',
 '\n',
 '\nGilbert and Gaillard\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n',
 '\n7+ \n',
 '\n',
 '\nDe Grote Hamersma\n',
 '\n',
 '\n',
 '\n',
 '\n']

In [107]:
def retrieve_award_info(product_link, award_link, soup, data):
    text = soup.find_all(text = True)
    stripped_text = [i.strip() for i in text if i != "\n"][2:]
    n = 0
    while True:
        try:
            if n > len(stripped_text)-1:
                break
            if 'Oogstjaar' in stripped_text[n]:
                vintage = stripped_text[n]
                j = n + 4
                k = n + 3
                if j > len(stripped_text):
                    items = stripped_text[n+1:]
                else:
                    items = stripped_text[n+1:j]
            else:
                vintage = None
                j = n + 3
                k = n + 2
                if j > len(stripped_text):
                    items = stripped_text[n:]
                else:
                    items = stripped_text[n:j]
            with open("gall_awards.csv", "a", encoding = "utf-8-sig", newline = '') as csv_file:
                writer = csv.writer(csv_file, delimiter = ";")
                info = {
                'product_link' : product_link,
                'award_link' : award_link,
                'vintage' : vintage,
                'grade' : items[0],
                'contributor' : items[1]}
                data.append(info)
                print(len(data))
                writer.writerow(list(info.values()))
            n = k
        except:
            print(f"Error at {n}")
            break


In [76]:
t = ['t punten', 'asdf']
print(t)

['t punten', 'asdf']
